In [ ]:
import warnings

import matplotlib as mpl

from wildfires.analysis.plotting import *
from wildfires.data.cube_aggregation import *
from wildfires.data.datasets import *
from wildfires.utils import land_mask as get_land_mask
from wildfires.utils import match_shape

In [ ]:
warnings.filterwarnings("ignore", ".*Collapsing a non-contiguous coordinate.*")
mpl.rc("figure", figsize=(14, 6))

In [ ]:
monthly_obs, mean_obs, climatology_obs = prepare_selection(
    Datasets(CCI_BurnedArea_MERIS_4_1()).select_variables("fraction of observed area")
)

In [ ]:
land_mask = ~get_land_mask()
mean_obs.cube.data.mask |= match_shape(land_mask, mean_obs.cube.shape)

## Mean observed area fraction

In [ ]:
_ = cube_plotting(mean_obs.cube, title="Mean Fraction of Observed Area")

## Mean count where observed area fraction >= 80%

In [ ]:
min_thres = 0.8
observed_counts = (monthly_obs.cube.data >= min_thres).astype("float64")
observed_counts.mask |= match_shape(land_mask, observed_counts.shape)
mean_obs_counts = np.ma.mean(observed_counts, axis=0)

In [ ]:
_ = cube_plotting(mean_obs_counts, title="Mean Observed Counts >= 80%")

## Before-after comparison of burned area

In [ ]:
mean_ba = prepare_selection(
    Datasets(CCI_BurnedArea_MERIS_4_1()).select_variables("CCI MERIS BA"), which="mean"
).cube
monthly_vars = prepare_selection(
    Datasets(CCI_BurnedArea_MERIS_4_1()).select_variables(
        ("CCI MERIS BA", "fraction of observed area")
    ),
    which="monthly",
)
monthly_obs = monthly_vars.select_variables(
    "fraction of observed area", inplace=False
).cube
monthly_ba = monthly_vars.select_variables("CCI MERIS BA").cube

mean_ba.data.mask |= match_shape(land_mask, mean_ba.shape)
monthly_ba.data.mask |= match_shape(land_mask, monthly_ba.shape)

## Before applying observational fraction mask

In [ ]:
print(
    "Mean BA:",
    mean_ba.collapsed(
        ("latitude", "longitude"),
        iris.analysis.MEAN,
        weights=iris.analysis.cartography.area_weights(mean_ba),
    ).data,
)
_ = cube_plotting(mean_ba, log=True, title="Mean CCI MERIS BA")

## After applying observational fraction mask >= 80%

In [ ]:
min_thres = 0.8
observed_mask = monthly_obs.data < min_thres
monthly_ba_masked = deepcopy(monthly_ba)
monthly_ba_masked.data.mask |= observed_mask

mean_ba_masked = monthly_ba_masked.collapsed("time", iris.analysis.MEAN)

In [ ]:
print(
    "Mean BA:",
    mean_ba_masked.collapsed(
        ("latitude", "longitude"),
        iris.analysis.MEAN,
        weights=iris.analysis.cartography.area_weights(mean_ba_masked),
    ).data,
)
_ = cube_plotting(mean_ba_masked, log=True, title="Mean Masked CCI MERIS BA >= 80% Obs")

### Histogram of data values in above plot - determine bin sizing from this?

In [ ]:
_ = plt.hist(
    mean_ba_masked.data.data.flatten()[~mean_ba_masked.data.mask.flatten()], bins=100
)
plt.yscale("log")